In [1]:
import numpy as np
import os
import pandas
from typing import Dict

## Descrição do problema

Determine a temperatura uniforme de equilíbrio $T_{placa}$ de uma placa vertical (altura $3.0m$; comprimento $6.0m$). Uma de suas faces (e) é exposta ao ar ambiente $T_{\infty}=310K$ e recebe um fluxo prescrito $q_e”=800\frac{W}{m^2}$, enquanto a outra face (d) recebe um fluxo prescrito $q_d”=140\frac{W}{m^2}$.

In [ ]:
plate_dimensions : Dict = {'height' : 3.0, 'lenght' : 6.0}
prescribed_heat : Dict = {'side_e' : 800, 'side_d' : 140}
enviroment_temperature : np.float128 = 310.0

## Modelagem do Problema
### Balanço de Energia
Como descrito no problema, a temperatura de equílibrio uniforme foi solicitada, isso significa que a energia armazenada no sistema deve ser igual a zero. Somando isso ao fato que não foi explicitado no problema que a placa gera algum tipo de calor temos que o balanço de enrgia pode ser descrito da seguinte forma:
$$ E_{entra} - E_{sai} = 0$$
Sendo que:
$$ E_{entra} = q_{e}"\cdot A + q_{d}"\cdot A = \left(q_{e}" + q_{d}"\right)\cdot A $$
$$ E_{sai} = h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) $$
Logo temos  que :
$$ \left(q_{e}" + q_{d}"\right)\cdot A - h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) = 0 $$
Como o valor solicitado é a a temperatura da placa tem-se que :
$$ T_{placa} = \frac{q_{e}" + q_{d}"}{h_{c}} + T_{\infty} $$
Fazendo a substituição dos valores conhecidos temos que :
$$ T_{placa} = \frac{800 + 140}{h_{c}} + 310 $$
Temos então que o resultado pode ser descrito como sendo :
$$ T_{placa} = \frac{940}{h_{c}} + 310 $$


In [ ]:
class Plate:
    def __init__(self,height : np.float128, lenght : np.float128) -> None:
        self.__dimensions__ = {'height':height, 'lenght':lenght}
    
    @property
    def area(self) -> np.float128:
        return self.__dimensions__['height']*self.__dimensions__['lenght']
    
    @property
    def height(self) -> np.float128:
        return self.__dimensions__['height']
    
    @property
    def lenght(self) -> np.float128:
        return self.__dimensions__['lenght']
